Решить методами Гаусса и Зейделя, найти $\lambda_{min}$ и $\lambda_{max}$, определить число обусловленности матрицы $\mu = \|A\| \cdot \|A^{-1}\|$. Сделать печать невязок обоих методов. Указать критерий останова итераций метода Зейделя.

In [1]:
import numpy as np

def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

a = [1] * 19
b = [-2] * 20
c = [1] * 19

A = tridiag(a, b, c)
A[0] = [1] + [0]*19
A[-1] = [1] + [2]*18 + [1]

b = np.array([1] + [2/i**2 for i in range(2, 21)])

print(A)
print(A.shape)

[[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 -2  1  0

## Метод Гаусса

In [2]:
def gaussian_elimination(A, b):
    n = len(A)
    
    if len(b) != n:
        raise ValueError("Различаются размеры A и b", n, len(b))
        
    for first_row in range(n - 1):
        for row in range(first_row + 1, n):
            multiplier = A[row][first_row] / A[first_row][first_row]
            A[row][first_row] = multiplier
            for col in range(first_row + 1, n):
                A[row][col] = A[row][col] - multiplier*A[first_row][col]
            b[row] = b[row] - multiplier*b[first_row]

    x = np.zeros(n)
    k = n - 1
    x[k] = b[k] / A[k, k]
    
    while k >= 0:
        x[k] = (b[k] - np.dot(A[k, k+1:], x[k+1:])) / A[k, k]
        k = k - 1

    return x

In [3]:
x = gaussian_elimination(A.copy(), b.copy())
error = np.dot(A, x) - b
print(f"Невязка: {error}")

Невязка: [  0.00000000e+00   0.00000000e+00   1.03272809e+00  -2.22044605e-16
   6.93889390e-17   2.77555756e-17   4.85722573e-17   0.00000000e+00
  -1.11022302e-16   1.28369537e-16  -5.55111512e-17   6.93889390e-18
   5.20417043e-18  -9.88792381e-17   5.72458747e-17   0.00000000e+00
   1.05818132e-16   2.77555756e-17  -5.29090660e-17   1.66967135e-15]


## Метод Зейделя

In [4]:
ITERATION_LIMIT = 1000

# Критерием останова будет незначительное отличие ответа текущей итерации
# от ответа предыдущей итерации. Возьмём например (10^-10).
x = np.zeros_like(b)
for j in range(1, ITERATION_LIMIT):
    x_new = np.zeros_like(x)
    
    for i in range(A.shape[0]):
        s1 = np.dot(A[i, :i], x_new[:i])
        s2 = np.dot(A[i, i + 1:], x[i + 1:])
        x_new[i] = (b[i] - s1 - s2) / A[i, i]

    if np.allclose(x, x_new, rtol=1e-10):
        print('Потребовалось ', j, ' итераций')
        break
    x = x_new

error = np.dot(A, x) - b
print(f"Невязка: {error}")

Потребовалось  180  итераций
Невязка: [  0.00000000e+00  -9.18734422e-10  -6.14777562e-10   2.20713670e-10
   1.48175856e-09   2.98170469e-09   4.48008990e-09   5.71641845e-09
   6.44691656e-09   6.47996125e-09   5.70592334e-09   4.11767948e-09
   1.81897367e-09  -9.80902902e-10  -3.98612940e-09  -6.84549306e-09
  -9.19255561e-09  -1.06905355e-08  -1.10767536e-08  -1.06685494e-16]


## Собственные значения и число обусловленности

In [5]:
print(np.linalg.eigvals(A))

[ 1.7963219  -3.97272261 -3.89450294 -3.7589475  -3.59033389 -3.35456314
 -3.12333733 -2.80339085 -2.54944428 -2.16515869 -1.93766382 -0.10715371
 -0.10836552 -0.40444485 -0.42171898 -1.50902903 -1.35380547 -0.83563561
 -0.90610368  1.        ]


$\lambda_{max} \approx 1.79632190314,\ \lambda_{min} \approx -3.97272260681$

In [6]:
print('Число обусловленности: ', np.linalg.cond(A))

Число обусловленности:  238.725557643


Получили, что наша задача не очень хорошо обусловлена. В частности из-за этого решения полученные 2 алгоритмами сильно отличаются.

In [7]:
print('Метод Гаусса: ', gaussian_elimination(A.copy(), b.copy()))
print('Метод Зейделя: ', x)

Метод Гаусса:  [ 1.         -0.78272809 -2.06545618 -2.09323396 -1.99601174 -1.81878952
 -1.58601174 -1.31241763 -1.00757353 -0.67803807 -0.3285026   0.03756178
  0.41751506  0.80930266  1.21129433  1.6221749   2.04086797  2.46648145
  2.89826777  3.33559426]
Метод Зейделя:  [ 1.          0.04402652 -0.41194696 -0.64569822 -0.75444948 -0.78320074
 -0.75639644 -0.68877581 -0.58990517 -0.46634317 -0.32278116 -0.16269022
  0.01128961  0.19710376  0.393122    0.59802912  0.81074873  1.03038875
  1.2562016   1.4875546 ]
